# Neuron set examples

In [1]:
import obi

### __Initialization:__ Loading a circuit

In [2]:
circuit = obi.Circuit(name="ToyCircuit-S1-6k", path="/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json")
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")

Circuit 'ToyCircuit-S1-6k' with 5924 neurons and 568717 synapses


### __Example 1:__ Adding node set dict to an existing SONATA circuit object

In [20]:
# Get SONATA circuit object
c = circuit.sonata_circuit
print("..." + str(c.node_sets.content)[-25:])

# Adding a node set to the circuit
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})
print("..." + str(c.node_sets.content)[-55:])

# Adding a node set with an exising name => NOT POSSIBLE
# obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})  # AssertionError: Node set 'Layer23' already exists!

# Update/overwrite an existing node set
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": ["Layer2", "Layer3"]}, overwrite_if_exists=True)  # Update/overwrite
print("..." + str(c.node_sets.content)[-58:])

# Add node set from NeuronSet object
neuron_set = obi.BasicNeuronSet(name="Layer123", circuit=circuit, population="All", node_sets=["Layer1", "Layer2", "Layer3"])
obi.NeuronSet.add_node_set_to_circuit(c, neuron_set.get_node_set_dict())
print("..." + str(c.node_sets.content)[-102:])

..., 'Layer6': {'layer': 6}}
..., 'Layer6': {'layer': 6}, 'Layer23': {'layer': [2, 3]}}
..., 'Layer6': {'layer': 6}, 'Layer23': ['Layer2', 'Layer3']}
..., 'Layer6': {'layer': 6}, 'Layer23': ['Layer2', 'Layer3'], 'Layer123': ['Layer1', 'Layer2', 'Layer3']}


### __Example 2:__ Use of different NeuronSet types

#### (a) __BasicNeuronSet__, based on existing (named) node sets

In [28]:
neuron_set1 = obi.BasicNeuronSet(name="Basic", circuit=circuit, population="All", node_sets=["Layer1", "Layer2", "Layer3"], random_sample=None)
print(f"NeuronSet '{neuron_set1.name}' of population '{neuron_set1.population}' with {neuron_set1.size} neurons:")
print(f"> Neuron IDs: {neuron_set1.get_ids(with_population=False)}")
print(f"> Node set dict: {neuron_set1.get_node_set_dict()}")

NeuronSet 'Basic' of population 'All' with 1659 neurons:
> Neuron IDs: [   0    1    2 ... 1656 1657 1658]
> Node set dict: {'Basic': ['Layer1', 'Layer2', 'Layer3']}


#### (b) __BasicNeuronSet__, based on existing (named) node sets, with random sub-sampling
<u>Note</u>: `random_sample` can be an absolute number or fraction

In [29]:
neuron_set2 = obi.BasicNeuronSet(name="BasicRandom", circuit=circuit, population="All", node_sets=["Layer1", "Layer2", "Layer3"], random_sample=10, random_seed=0)
print(f"NeuronSet '{neuron_set2.name}' of population '{neuron_set2.population}' with {neuron_set2.size} neurons:")
print(f"> Neuron IDs: {neuron_set2.get_ids(with_population=False)}")
print(f"> Node set dict: {neuron_set2.get_node_set_dict()}")

NeuronSet 'BasicRandom' of population 'All' with 10 neurons:
> Neuron IDs: [ 115  145  215  355  378  568 1004 1194 1219 1252]
> Node set dict: {'BasicRandom': {'population': 'All', 'node_id': [115, 145, 215, 355, 378, 568, 1004, 1194, 1219, 1252]}}


#### (c) __IDNeuronSet__, based on individual neuron IDs

In [41]:
neuron_set3 = obi.IDNeuronSet(name="IDSet", circuit=circuit, population="All", neuron_ids=[1, 2, 3])
print(f"NeuronSet '{neuron_set3.name}' of population '{neuron_set3.population}' with {neuron_set3.size} neurons:")
print(f"> Neuron IDs: {neuron_set3.get_ids(with_population=False)}")
print(f"> Node set dict: {neuron_set3.get_node_set_dict()}")

NeuronSet 'IDSet' of population 'All' with 3 neurons:
> Neuron IDs: [1 2 3]
> Node set dict: {'IDSet': {'population': 'All', 'node_id': [1, 2, 3]}}


#### (d) __IDNeuronSet__, based on individual neuron IDs, with random sub-sampling
<u>Note</u>: `random_sample` can be an absolute number or fraction

In [42]:
neuron_set4 = obi.IDNeuronSet(name="IDSetRandom", circuit=circuit, population="All", neuron_ids=range(10), random_sample=0.5, random_seed=999)
print(f"NeuronSet '{neuron_set4.name}' of population '{neuron_set4.population}' with {neuron_set4.size} neurons:")
print(f"> Neuron IDs: {neuron_set4.get_ids(with_population=False)}")
print(f"> Node set dict: {neuron_set4.get_node_set_dict()}")

NeuronSet 'IDSetRandom' of population 'All' with 5 neurons:
> Neuron IDs: [1 3 4 7 9]
> Node set dict: {'IDSetRandom': {'population': 'All', 'node_id': [1, 3, 4, 7, 9]}}


#### (d) __PropertyNeuronSet__, based on neuron properties

In [54]:
neuron_set5 = obi.PropertyNeuronSet(name="L23I", circuit=circuit, population="All", property_specs={"layer": [2, 3], "synapse_class": "INH"})
print(f"NeuronSet '{neuron_set5.name}' of population '{neuron_set5.population}' with {neuron_set5.size} neurons:")
print(f"> Neuron IDs: {neuron_set5.get_ids(with_population=False)}")
print(f"> Node set dict: {neuron_set5.get_node_set_dict()}")

# Optional: Individual neuron IDs resolved
print(f"> Node set dict with IDs resolved [OPTIONAL]: {neuron_set5.get_node_set_dict(force_resolve_ids=True)}")

NeuronSet 'L23I' of population 'All' with 200 neurons:
> Neuron IDs: [ 70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87
  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105
 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141
 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159
 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177
 178 179 180 181 182 183 184 185 186 187 188 788 789 790 791 792 793 794
 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812
 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830
 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848
 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866
 867 868]
> Node set dict: {'L23I': {'layer': [2, 3], 'synapse_class': 'INH'}}
> Node set dict with IDs resolved [OPTIONAL]: {'L

#### (e) __PropertyNeuronSet__, based on neuron properties, combined with exising (named) node sets
<u>Note</u>: Individual neuron IDs will be resolved since a combination of properties and node sets is not possible otherwise!

In [46]:
neuron_set6 = obi.PropertyNeuronSet(name="L23I", circuit=circuit, population="All", property_specs={"synapse_class": "INH"}, node_sets=["Layer2", "Layer3"])
print(f"NeuronSet '{neuron_set6.name}' of population '{neuron_set6.population}' with {neuron_set6.size} neurons:")
print(f"> Neuron IDs: {neuron_set6.get_ids(with_population=False)}")
print(f"> Node set dict: {neuron_set6.get_node_set_dict()}")

NeuronSet 'L23I' of population 'All' with 200 neurons:
> Neuron IDs: [ 70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87
  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105
 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141
 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159
 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177
 178 179 180 181 182 183 184 185 186 187 188 788 789 790 791 792 793 794
 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812
 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830
 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848
 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866
 867 868]
> Node set dict: {'L23I': {'population': 'All', 'node_id': [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84

### __Example 3:__ Writing a neuron set to a SONATA node set file
<u>Note</u>: The node sets file name is by default taken from the original circuit. An alternative name can optionally be provided.

<u>Note 2</u>: Overwrite (`overwrite_if_exists`) and append (`append_if_exists`) options exist.

In [10]:
output_path = "./"

# Write new file, overwrite if existing
neuron_set = obi.BasicNeuronSet(name="L123", circuit=circuit, population="All", node_sets=["Layer1", "Layer2", "Layer3"])
nset_file = neuron_set.write_node_set_file(output_path=output_path, overwrite_if_exists=True)

# Append to existing file, but name already exists => NOT POSSIBLE
# nset_file = neuron_set.write_node_set_file(output_path="./", append_if_exists=True)  # AssertionError: Appending not possible, node set 'Basic' already exists!

# Append to existing file
neuron_set = obi.BasicNeuronSet(name="L456", circuit=circuit, population="All", node_sets=["Layer4", "Layer5", "Layer6"])
nset_file = neuron_set.write_node_set_file(output_path=output_path, append_if_exists=True)

if os.path.exists(nset_file):
    print(f"Node set file: {nset_file}")

Node set file: ./node_sets.json
